In [5]:
import pandas as pd
import re 
import numpy as np
import urllib

In [6]:
# load list of titles as one column dataframe
df_raw = pd.read_csv("../2b_Organizer_extraction - SRA/conserved_titles.txt", low_memory = True, delimiter="\t", names=["title"])

In [7]:
# list of markers for detec sanger, nuclear, and chloroplast info
nuclear_markers = "Internal transcribed spacer|ITS1|ITS2|ETS|nrITS|18S"
chl_markers = "trn[LGR]|atpB|rubisco|rbcL|psbA|trnH|matk|psbJ|petA"

# set catagories of search
categories = {
    #scope
    "Taxonomy": "taxonom|delimit|clas*ific|circumpscri|new species|nueva.? especie",
    "Systematic": "s[yi]stem.t| comple[xj]|phylog|filog|clade|speciation|monophy|monofi|rela[tc]ion|biogeo|evolu",
    # "Other related": "barcod| barra|h[yi]brid",
    "Mexico": "m*[xj]ico",
    #tematics
    "Morphology": "morpholog|morfolog",
    "First gen.": f"{nuclear_markers}|{chl_markers}| sanger | marker| nuclear region| chloroplast region|regi.n.*? nucr|regi.n.*? cloro",
    "Second gen.": "radseq|angiosperm353|hybseq|454|illumina|iontorrent",
    "Third gen.": "pacbio|nanopore",
    "Nuclear info": f"nuclear|{nuclear_markers}",
    "Chloroplast info": f"c.?lorop|{chl_markers}",
    "Parsimony": "parsimon",
    "Maximum likelihood": " ML |maximum likelihood|verosimilitud",
    "Bayesian": "bayes",
    "MSC": " astral | astrid | bpp |coalescent"
    ""
}

In [8]:
test = "A Molecular Phylogenetic Study of the Palmae (Arecaceae) Based on atpB, rbcL, and 18S nrDNA Sequences"
for k in categories:
    print(f"{k}: {re.search(categories[k], test, re.IGNORECASE)}")

Taxonomy: None
Systematic: <re.Match object; span=(12, 18), match='Phylog'>
Mexico: None
Morphology: None
First gen.: <re.Match object; span=(66, 70), match='atpB'>
Second gen.: None
Third gen.: None
Nuclear info: <re.Match object; span=(82, 85), match='18S'>
Chloroplast info: <re.Match object; span=(66, 70), match='atpB'>
Parsimony: None
Maximum likelihood: None
Bayesian: None
MSC: None


In [9]:
# copy original df
df = df_raw.copy()

# create the main columns for the scope of this search and google link
df["REVISADO (bibtex citation)"] = np.nan
df["Google link"] = np.nan
df["Taxonomy"] = np.nan
df["Systematic"] = np.nan
# df["Other related"] = np.nan
df["Mexico"] = np.nan

# iterate row by row
for index, row in df.iterrows():
    # check every key in category dictionary 
    for k in categories:
        # create empty column
        # search every pattern in title
        if re.search(categories[k], row["title"], re.IGNORECASE):
           #add a helper link to go directly to google scholar
           searchquery = urllib.parse.urlencode({"q": row["title"]})
           df.loc[index, "Google link"] = f"https://scholar.google.com/scholar?&{searchquery}"
           #add Auto in the cell to indicate this part of the search is true
           df.loc[index, k] = "Auto"

In [10]:
df

,title,REVISADO (bibtex citation),Google link,Taxonomy,Systematic,Mexico,Second gen.,Third gen.,Chloroplast info,Morphology,First gen.,Nuclear info
0,cultivar:25e11-NonTol Genome sequencing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cultivar:25e11-Tol Genome sequencing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cultivar:Allahabad Safeda Genome sequencing a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cultivar:Bridgeton Genome sequencing and asse...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cultivar:CCC population Genome sequencing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4030,sweet potato starch content transcriptome,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4031,sweetpotato storage root Raw sequence reads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4032,transcriptome covering the whole development p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4033,transcriptomic of exogenous GABA in poplar(Pop...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# remove rows that has NaN for all categories (out of the scope of this study)
df_final = df[(df['Taxonomy'].notnull()) | (df['Systematic'].notnull()) | (df['Mexico'].notnull())]

In [16]:
df_final

,title,REVISADO (bibtex citation),Google link,Taxonomy,Systematic,Mexico,Second gen.,Third gen.,Chloroplast info,Morphology,First gen.,Nuclear info
10,1) 37 diploid Helianthus species with Phoebant...,NaN,https://scholar.google.com/scholar?&q=1%29+37+...,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,3D genome reorganizations in the evolution and...,NaN,https://scholar.google.com/scholar?&q=3D+genom...,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,A New Pipeline for Removing Paralogs in Target...,NaN,https://scholar.google.com/scholar?&q=A+New+Pi...,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,A VIEW OF RHYNCHOSPOREAE (CYPERACEAE) DIVERSIF...,NaN,https://scholar.google.com/scholar?&q=A+VIEW+O...,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,A backbone phylogeny of the American clade oak...,NaN,https://scholar.google.com/scholar?&q=A+backbo...,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3906,We included 85 samples of species of Bursera (...,NaN,https://scholar.google.com/scholar?&q=We+inclu...,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3923,Whole-Genome Duplication and Acceleration of M...,NaN,https://scholar.google.com/scholar?&q=Whole-Ge...,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3931,Widespread adaptive evolution during repeated ...,NaN,https://scholar.google.com/scholar?&q=Widespre...,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3992,double digest RAD data from Berberis spp. (Ber...,NaN,https://scholar.google.com/scholar?&q=double+d...,NaN,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df[(df['Mexico'].notnull()) & (df['Systematic'].isnull()) & (df['Taxonomy'].isnull())]

,title,REVISADO (bibtex citation),Google link,Taxonomy,Systematic,Mexico,Second gen.,Third gen.,Chloroplast info,Morphology,First gen.,Nuclear info
3691,Toxicodendron radicans,NaN,https://scholar.google.com/scholar?&q=Toxicode...,NaN,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN
3692,Toxicodendron radicans (poison ivy) leaf and r...,NaN,https://scholar.google.com/scholar?&q=Toxicode...,NaN,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN
3992,double digest RAD data from Berberis spp. (Ber...,NaN,https://scholar.google.com/scholar?&q=double+d...,NaN,NaN,Auto,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_final.to_csv("final_database_SRA.csv")

In [19]:
df_final.shape

(153, 12)